In [2]:
import sqlite3
from utils.fs import GetFilesIn
import json

ROOT = '/home/tony/workspace/python/grad/resources/eco_meta_cyc_25-5'
DB_PATH = f'{ROOT}/Biocyc-Parse/parsed/metacyc25-1.limes.psql'

In [22]:
import sys
SCRIPTS_PATH = '/home/tony/workspace/python/grad/resources/eco_meta_cyc_25-5/Biocyc-Parse/'
if SCRIPTS_PATH not in sys.path:
    sys.path.append(SCRIPTS_PATH)
import scripts.parseDat as par
import importlib
importlib.reload(par)

CON = sqlite3.connect(DB_PATH)
CUR = CON.cursor()

def registerTable(table:str, fields:list[str]):
    if len(list(CUR.execute(f"SELECT name FROM tables WHERE name = '{table}'"))) == 0:
        CUR.executemany(f"INSERT INTO tables (name, field) VALUES (?, ?)", [(table, f) for f in fields])

def setup():
    make_tables = [
        '''CREATE TABLE IF NOT EXISTS tables (
            name TEXT NOT NULL,
            field TEXT NOT NULL,
            PRIMARY KEY (name, field)
        )''',
        '''CREATE TABLE IF NOT EXISTS json_keys_of_dats (
            key TEXT NOT NULL,
            table_name TEXT NOT NULL, 
            PRIMARY KEY (key, table_name),
            FOREIGN KEY (table_name) REFERENCES tables (name) 
        )''',
    ]
    for cmd in make_tables:
        CUR.execute(cmd)

    registerTable('json_keys_of_dats', ['key', 'table_name'])
setup()

def parseDats():
    DATA_DIR = '/home/tony/workspace/python/grad/resources/eco_meta_cyc_25-5/meta/25.1/data'

    DATS = [
        'proteins.dat',
        'enzrxns.dat',
        'pathways.dat',
        'genes.dat',
        'regulation.dat',
        'reactions.dat',
        'compounds.dat',
        'protein-features.dat',
        'protligandcplxes.dat',
        'rnas.dat',
        'species.dat',
    ]

    all_dats = {}

    for dat in DATS:
        print(f"{dat}")
        
        pdat = par.parse(f'{DATA_DIR}/{dat}', 'UNIQUE-ID', {
            'DBLINKS': lambda x: x[1:-1].replace('"', '').split(' ')[:2],
            'GIBBS-0': lambda x: x.strip(),
            'MOLECULAR-WEIGHT-EXP': lambda x: x.strip(),
        }, all_fields=True)

        dat_name = dat.replace('.', '_').replace('-', '_')
        CUR.execute(f'''CREATE TABLE IF NOT EXISTS {dat_name} (
            id TEXT PRIMARY KEY,
            json TEXT NOT NULL
        )''',)
        registerTable(dat_name, ['id', 'json'])

        # just using a dict because not enough useful links between tables to justify
        entries: list[tuple] = []
        json_keys = set()
        for k, val in pdat.items():
            entries.append((k, json.dumps(val, separators=(',', ':'))))
            json_keys = json_keys.union(set(val.keys()))

        CUR.executemany(f"INSERT INTO {dat_name} (id, json) VALUES (?, ?)", entries)
        CUR.executemany(f"INSERT INTO json_keys_of_dats (key, table_name) VALUES (?, ?)", [(dat_name, k) for k in json_keys])
        all_dats[dat_name] = pdat

    # ==========================================================
    # make reverse mappings, like aws dynamo's secondary indexes

    def dictAppend(d:dict, k, v):
        if k in d:
            arr = d[k]
            arr.append(v)
        else:
            d[k] = [v]

    def makeRev(mapping, revKey, parser):
        rev = {}
        for k, v in mapping.items():
            if revKey not in v: continue
            for val in v[revKey]:
                parsed = parser(val)
                if type(parsed) == list:
                    [dictAppend(rev, p, k) for p in parsed]
                else:
                    dictAppend(rev, parsed, k)
        return rev

    def linearize(mapping):
        entries = set()
        for k, vs in mapping.items():
            for v in vs:
                if type(k) == tuple:
                    db, id = k
                    entries.add((db, id, v))
                else:
                    entries.add((k, v))
        return entries

    # seperate from loop below since key is db + id (where id is likely db accession)
    print('dblink mapping')
    db_mapping = makeRev(all_dats['proteins_dat'], 'DBLINKS', lambda x: tuple(x))
    CUR.execute(f'''CREATE TABLE IF NOT EXISTS dblinks_proteins_map (
        db TEXT NOT NULL,
        id TEXT NOT NULL,
        found_in TEXT NOT NULL,
        PRIMARY KEY (db, id, found_in),
        FOREIGN KEY (found_in) REFERENCES proteins_dat (id)
    )''',)
    registerTable('dblinks_proteins_map', ['db', 'id', 'found_in'])
    CUR.executemany('INSERT INTO dblinks_proteins_map (db, id, found_in) VALUES (?, ?, ?)', linearize(db_mapping))

    # source table, mapping name, target id, target table
    mappings = [
        ('enzrxns_dat',     'protein_enzrxn_map',   'ENZYME',               'proteins_dat'),
        ('reactions_dat',   'enzrxn_reaction_map',  'ENZYMATIC-REACTION',   'enzrxns_dat'),
        ('pathways_dat',    'reaction_pathway_map', 'REACTION-LIST',        'reactions_dat'),
    ]

    for source, name, target_id, target in mappings:
        print(f'mapping {name}')
        mapping = makeRev(all_dats[source], target_id, lambda x: x)
        CUR.execute(f'''CREATE TABLE IF NOT EXISTS {name} (
            id TEXT NOT NULL,
            found_in TEXT NOT NULL,
            PRIMARY KEY (id, found_in),
            FOREIGN KEY (found_in) REFERENCES {target} (id)
        )''',)
        registerTable(name, ['id', 'found_in'])

        CUR.executemany(f'INSERT INTO {name} (id, found_in) VALUES (?, ?)', linearize(mapping))
parseDats()

print('committing')
CON.commit()
print('done')
CON.close()

proteins.dat
enzrxns.dat
pathways.dat
genes.dat
regulation.dat
reactions.dat
compounds.dat
protein-features.dat
protligandcplxes.dat
rnas.dat
species.dat
dblink mapping
mapping protein_enzrxn_map
mapping enzrxn_reaction_map
mapping reaction_pathway_map
committing
done


In [19]:
CON.close()

In [3]:
CON = sqlite3.connect(DB_PATH)
CUR = CON.cursor()

In [4]:
list(CUR.execute("SELECT DISTINCT name from tables"))

[('compounds_dat',),
 ('dblinks_proteins_map',),
 ('enzrxn_reaction_map',),
 ('enzrxns_dat',),
 ('genes_dat',),
 ('json_keys_of_dats',),
 ('pathways_dat',),
 ('protein_enzrxn_map',),
 ('protein_features_dat',),
 ('proteins_dat',),
 ('protligandcplxes_dat',),
 ('reaction_pathway_map',),
 ('reactions_dat',),
 ('regulation_dat',),
 ('rnas_dat',),
 ('species_dat',)]

In [24]:
list(CUR.execute("SELECT field from tables where name='dblinks_proteins_map'"))

[('db',), ('found_in',), ('id',)]

In [6]:
for x in CUR.execute("SELECT id from reactions_dat"):
    break
x

('+-BORNEOL-DEHYDROGENASE-RXN',)

In [14]:
x = list(CUR.execute("""
select r.id, rp.id, rp.found_in
from reactions_dat as r inner join reaction_pathway_map as rp on r.id = rp.id
where r.id ='+-BORNEOL-DEHYDROGENASE-RXN'
"""))

In [15]:
for xx in x:
    print(xx)

('+-BORNEOL-DEHYDROGENASE-RXN', '+-BORNEOL-DEHYDROGENASE-RXN', 'PWY-6990')
